In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
import json
import itertools
from tqdm import tqdm

In [ ]:
def rossler_derivatives(state, t, a=0.2, b=0.2, c=5.7):
    """Compute time derivatives [dx/dt, dy/dt, dz/dt] for the Rössler system."""
    x, y, z = state
    dxdt = -y - z
    dydt = x + a * y
    dzdt = b + z * (x - c)
    return [dxdt, dydt, dzdt]

def generate_rossler_data(
    initial_state=[1.0, 0.0, 0.0],
    tmax=25.0,
    dt=0.01,
    a=0.2,
    b=0.2,
    c=5.7
):
    """
    Numerically integrate Rössler equations x'(t), y'(t), z'(t) using odeint.
    Returns:
       t_vals: array of time points
       sol   : array shape [num_steps, 3] of [x(t), y(t), z(t)]
    """
    num_steps = int(tmax / dt)
    t_vals = np.linspace(0, tmax, num_steps)
    sol = odeint(rossler_derivatives, initial_state, t_vals, args=(a, b, c))
    return t_vals, sol

In [ ]:
def compute_valid_prediction_time(y_true, y_pred, t_vals, threshold, lambda_max, dt):
    """
    Compute the Valid Prediction Time (VPT) and compare it to Lyapunov time T_lambda = 1 / lambda_max.
    
    Parameters
    ----------
    y_true : ndarray of shape (N, dim)
        True trajectory over time.
    y_pred : ndarray of shape (N, dim)
        Model's predicted trajectory over time (closed-loop).
    t_vals : ndarray of shape (N,)
        Time values corresponding to the trajectory steps.
    threshold : float, optional
        The error threshold, default is 0.4 as in your snippet.
    lambda_max : float, optional
        Largest Lyapunov exponent. Default=0.9 for Lorenz.
        
    Returns
    -------
    T_VPT : float
        Valid prediction time. The earliest time at which normalized error surpasses threshold
        (or the last time if never surpassed).
    T_lambda : float
        Lyapunov time = 1 / lambda_max
    ratio : float
        How many Lyapunov times the model prediction remains valid, i.e. T_VPT / T_lambda.
    """
    # 1) Average of y_true
    y_mean = np.mean(y_true, axis=0)  # shape (dim,)
    
    # 2) Time-averaged norm^2 of (y_true - y_mean)
    y_centered = y_true - y_mean
    denom = np.mean(np.sum(y_centered**2, axis=1))  # scalar
    
    # 3) Compute the normalized error delta_gamma(t) = ||y_true - y_pred||^2 / denom
    diff = y_true - y_pred
    err_sq = np.sum(diff**2, axis=1)  # shape (N,)
    delta_gamma = err_sq / denom      # shape (N,)
    
    # 4) Find the first time index where delta_gamma(t) exceeds threshold
    idx_exceed = np.where(delta_gamma > threshold)[0]
    if len(idx_exceed) == 0:
        # never exceeds threshold => set T_VPT to the final time
        T_VPT = t_vals[-1]
    else:
        T_VPT = t_vals[idx_exceed[0]]
    
    # 5) Compute T_lambda and ratio
    T_lambda = 1.0 / lambda_max

    # print(f"\n--- Valid Prediction Time (VPT) with threshold={threshold}, lambda_max={lambda_max} ---")

    T_VPT = (T_VPT - t_vals[0])  # Adjust T_VPT to be relative to the start time
    ratio = T_VPT / T_lambda

    return T_VPT, T_lambda, ratio

In [ ]:
def scale_spectral_radius(W, target_radius=0.95):
    """
    Scales a matrix W so that its largest eigenvalue magnitude = target_radius.
    """
    eigvals = np.linalg.eigvals(W)
    radius = np.max(np.abs(eigvals))
    if radius == 0:
        return W
    return (W / radius) * target_radius

def augment_state_with_squares(x):
    """
    Given state vector x in R^N, return [ x, x^2, 1 ] in R^(2N+1).
    We'll use this for both training and prediction.
    """
    x_sq = x**2
    return np.concatenate([x, x_sq, [1.0]])  # shape: 2N+1

class CRJ3D:
    """
    Cycle Reservoir with Jumps (CRJ) for 3D->3D single-step tasks.
    We form a ring adjacency with an extra 'jump' edge in each row.
    This can help capture multiple timescales or delayed memory
    while retaining the easy ring structure.

    The adjacency is built as follows (reservoir_size = mod N):
      For each i in [0..N-1]:
        W[i, (i+1) % mod N] = 1.0
        W[i, (i+jump) % mod N] = 1.0
    Then we scale by 'spectral_radius.' We do an ESN update
    with readout [ x, x^2, 1 ] -> next step in R^3.
    """

    def __init__(self,
                 reservoir_size=300,
                 jump=10,                # offset for the jump
                 spectral_radius=0.95,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 seed=42):
        """
        reservoir_size: how many nodes in the ring
        jump            : the offset for the 2nd connection from node i
        spectral_radius : scale adjacency
        input_scale     : scale factor for W_in
        leaking_rate    : ESN 'alpha'
        ridge_alpha     : ridge penalty for readout
        seed            : random seed
        """
        self.reservoir_size = reservoir_size
        self.jump = jump
        self.spectral_radius = spectral_radius
        self.input_scale = input_scale
        self.leaking_rate = leaking_rate
        self.ridge_alpha = ridge_alpha
        self.seed = seed

        # build adjacency
        np.random.seed(self.seed)
        W = np.zeros((reservoir_size, reservoir_size))
        for i in range(reservoir_size):
            # cycle edge: i -> (i+1)%N
            W[i, (i+1) % reservoir_size] = 1.0
            # jump edge: i -> (i+jump)%N
            W[i, (i + self.jump) % reservoir_size] = 1.0

        # scale spectral radius
        W = scale_spectral_radius(W, self.spectral_radius)
        self.W = W

        # input weights => shape [N,3]
        np.random.seed(self.seed+100)
        W_in = (np.random.rand(reservoir_size, 3) - 0.5)*2.0*self.input_scale
        self.W_in = W_in

        # readout
        self.W_out = None
        self.x = np.zeros(self.reservoir_size)

    def reset_state(self):
        self.x = np.zeros(self.reservoir_size)

    def _update(self, u):
        """
        Single-step ESN update:
          x(t+1) = (1-alpha)*x(t) + alpha*tanh( W x(t) + W_in u(t) )
        """
        pre_activation = self.W @ self.x + self.W_in @ u
        x_new = np.tanh(pre_activation)
        alpha = self.leaking_rate
        self.x = (1.0 - alpha)*self.x + alpha*x_new

    def collect_states(self, inputs, discard=100):
        """
        Teacher forcing => feed the real 3D inputs => gather states.
        Return (states_after_discard, states_discarded).
        """
        self.reset_state()
        states = []
        for val in inputs:
            self._update(val)
            states.append(self.x.copy())
        states = np.array(states)
        return states[discard:], states[:discard]

    def fit_readout(self, train_input, train_target, discard=100):
        """
        gather states => polynomial readout => solve ridge
        """
        states_use, _ = self.collect_states(train_input, discard=discard)
        target_use = train_target[discard:]
        X_list = []
        for s in states_use:
            # polynomial expansion => [ x, x^2, 1 ]
            X_list.append(augment_state_with_squares(s))
        X_aug = np.array(X_list)

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, target_use)
        self.W_out = reg.coef_  # shape => (3, 2N+1)

    def predict_autoregressive(self, initial_input, n_steps):
        """
        fully autoregressive => feed last output => next input
        """
        preds = []
        #self.reset_state()
        current_in = np.array(initial_input)
        for _ in range(n_steps):
            self._update(current_in)
            big_x = augment_state_with_squares(self.x)
            out = self.W_out @ big_x  # shape => (3,)
            preds.append(out)
            current_in = out
        return np.array(preds)
        
    def predict_open_loop(self, test_input):
        preds = []
        for true_input in test_input:
            self._update(true_input)
            x_aug = augment_state_with_squares(self.x)
            out = self.W_out @ x_aug
            preds.append(out)
        return np.array(preds)

In [ ]:
grid = {
    "spectral_radius": [0.92, 0.95, 0.98],
    "jump": [5, 10, 15, 20],
    "input_scale": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    "ridge_alpha": [1e-7],
    "leaking_rate": [0.1, 0.3, 0.5, 0.7, 0.9],
}

In [ ]:
def run_grid_search(model_class, param_grid, model_name,
                    output_path="grid_search_results.json"):
    # Precompute param combinations
    combos = list(itertools.product(*param_grid.values()))
    param_keys = list(param_grid.keys())
    print(f"\n== Initial grid search for {model_name} with {len(combos)} combinations ==")

    results = []
    # tqdm adds a progress bar for better visualization
    for comb in tqdm(combos, desc="Grid Search"):
        params = dict(zip(param_keys, comb))
        seed_scores = []
        
        # Run all 20 seeds
        for initial_state in [[1.0,1.0,1.0],[1.0,2.0,3.0],[2.0,1.5,4.0]]:
            tmax = 250
            dt   = 0.02
            t_vals, rossler_traj = generate_rossler_data(
                initial_state=initial_state,
                tmax=tmax,
                dt=dt
            )
            
            washout = 2000
            t_vals = t_vals[washout:]
            rossler_traj = rossler_traj[washout:]
            
            # normalize
            scaler = MinMaxScaler()
            scaler.fit(rossler_traj)
            rossler_traj = scaler.transform(rossler_traj)
            
            T_data = len(rossler_traj)
            for train_frac in [0.7,0.75,0.8]:
                train_end = int(train_frac*(T_data-1))
                train_input  = rossler_traj[:train_end]
                train_target = rossler_traj[1:train_end+1]
                test_input   = rossler_traj[train_end:-1]
                test_target  = rossler_traj[train_end+1:]
                n_test_steps = len(test_input)
                initial_in = test_input[0]
                for seed in np.arange(1,6):
                    model = model_class(**params, seed=seed)
                    model.fit_readout(train_input, train_target, discard=100)
                    preds = model.predict_autoregressive(initial_in, n_test_steps)
                    T_VPT_s, _, _ = compute_valid_prediction_time(test_target,preds,t_vals,0.4,0.9,dt)
                    seed_scores.append(T_VPT_s)
        mean_score = float(np.mean(seed_scores))
        std_dev    = float(np.std(seed_scores))
        # is_stable  = std_dev < 1.5
        # status     = "Stable" if is_stable else "Unstable"
        
        # print(f"Params: {params} → Avg T_VPT={mean_score:.3f}, "
        #       f"Std Dev={std_dev:.3f} → {status}")

        results.append({
            "params":      params,
            "seed_scores": seed_scores,
            "mean_T_VPT":  mean_score,
            "std_dev":     std_dev,
            # "stable":      is_stable
        })

    # Save results
    with open(output_path, "w") as f:
        json.dump(results, f, indent=2)
    print(f"\nAll results saved to `{output_path}`")
    
    return results

In [ ]:
run_grid_search(CRJ3D, grid, "CRJ", output_path="crj_grid_search_results.json")